<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelBinarizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [7]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
logit_valid_pred

In [5]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67956914653526068

In [12]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [4]:
X_train, y_train = train.iloc[:,:-1], train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_train['route']=X_train['Origin']+'-'+X_train['Dest']
X_train=X_train.drop(['Origin','Dest'],axis=1)

In [5]:
for c in ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier','route']:
    X_train[c].astype('category')

In [109]:
scaler = StandardScaler()
X_train['DepTime']=scaler.fit_transform(X_train[['DepTime']])
X_train['Distance']=scaler.fit_transform(X_train[['Distance']])

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
Month            100000 non-null object
DayofMonth       100000 non-null object
DayOfWeek        100000 non-null object
DepTime          100000 non-null int64
UniqueCarrier    100000 non-null object
Distance         100000 non-null int64
route            100000 non-null object
dtypes: int64(2), object(5)
memory usage: 5.3+ MB


In [118]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [7]:
X_test=test
X_test['route']=X_test['Origin']+'-'+X_test['Dest']
X_test=X_test.drop(['Origin','Dest'],axis=1)
for c in ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier','route']:
    X_test[c].astype('category')

In [115]:
scaler = StandardScaler()
X_test['DepTime']=scaler.fit_transform(X_test[['DepTime']])
X_test['Distance']=scaler.fit_transform(X_test[['Distance']])

In [8]:
X_test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Distance,route
0,c-7,c-25,c-3,615,YV,598,MRY-PHX
1,c-4,c-17,c-2,739,WN,1235,LAS-HOU
2,c-12,c-2,c-7,651,MQ,577,GSP-ORD
3,c-3,c-25,c-7,1614,WN,377,BWI-MHT
4,c-6,c-6,c-3,1505,UA,258,ORD-STL


In [44]:
'''empt=pd.DataFrame()
for field in ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier','route']:
    lb,temp=None,None
    lb=LabelBinarizer()
    temp=lb.fit_transform(train[field])
    temp_pd=pd.DataFrame(temp,columns=[field+str(i) for i in range(len(lb.classes_))])
    empt=pd.concat([empt,temp_pd], axis=1)'''

"empt=pd.DataFrame()\nfor field in ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier','route']:\n    lb,temp=None,None\n    lb=LabelBinarizer()\n    temp=lb.fit_transform(train[field])\n    temp_pd=pd.DataFrame(temp,columns=[field+str(i) for i in range(len(lb.classes_))])\n    empt=pd.concat([empt,temp_pd], axis=1)"

In [6]:
scaler = StandardScaler()
first_part = pd.DataFrame(scaler.fit_transform(train[['DepTime','Distance']]))
result=pd.concat([first_part,empt],axis=1)

In [23]:
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [9]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(result, y_train, test_size=0.3, random_state=17)

In [82]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67898882018079865

In [10]:
xgb=XGBClassifier(silent=True)


In [11]:
params={'max_depth':range(2,10)}
cv=GridSearchCV(xgb,params,n_jobs=-1,scoring='roc_auc')
cv.fit(X_train_part, y_train_part)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(2, 10)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='roc_auc', verbose=0)

In [ ]:
cv.best_params_

In [26]:
X_train_part

array([[ 559, 1148],
       [ 612,  645],
       [ 102,  525],
       ..., 
       [ 187,  948],
       [1400, 1402],
       [ 293,  913]], dtype=int64)

In [9]:
model = CatBoostClassifier(random_seed=17)

In [10]:
model.fit(X_train, y_train, cat_features=[0,1,2,4,6])

In [122]:
preds_class = model.predict_proba(X_valid)
roc_auc_score(y_valid, preds_class[:,1])

0.78105955576317687

In [11]:
test_pred = model.predict_proba(X_test)[:,1]

pd.Series(test_pred, name='dep_delayed_15min').to_csv('new_cat_2feat.csv', index_label='id', header=True)